In [185]:
def load_gnbcucp_cm(gnbcucp_filepath):
    df = open_file(gnbcucp_filepath)
    df['GNBID']= df['GNBID'].map(str)
    df['GNBID_LENGTH']= df['GNBID_LENGTH'].map(str)
    df['MCC']= df['MCC'].map(str)
    df['MNC']= df['MNC'].map(str)

    return df

In [186]:
def load_nrcellcu_cm(nrcellcu_filepath):
    df = open_file(nrcellcu_filepath)
    df['cellLocalId']= df['cellLocalId'].map(str)
    df['nCI']= df['nCI'].map(str)

    return df

In [187]:
def load_merge_gnbcucp_cellcu(gnbcucp_filepath, cellcu_filepath):
    df_gnbcucp = load_gnbcucp_cm(gnbcucp_filepath)
    df_nrcelldu = load_nrcellcu_cm(cellcu_filepath)

    df_cucp_cellcu_merged = df_nrcelldu.merge(
    df_gnbcucp, how="left", left_on=['meFdn', 'parentRefId'], right_on=['meFdn', 'refId'])

    df_cucp_cellcu_merged.rename({'refId_x': 'refId'}, axis=1, inplace=True)
    df_cucp_cellcu_merged.drop(columns=['refId_y'], inplace=True)

    df_cucp_cellcu_merged['NodeName'] = df_cucp_cellcu_merged['meFdn'].apply(lambda x: split_name(x, 'ManagedElement='))
    df_cucp_cellcu_merged['CellName'] = df_cucp_cellcu_merged['refId'].apply(lambda x: split_name(x, 'NRCellCU='))

    return df_cucp_cellcu_merged


In [188]:
def load_nrcellcu_ebs(nrcellcu_ebs_filepath, date, counters):
    df = open_file(nrcellcu_ebs_filepath)
    df = df[df['rdiTimeStamp'] == date]

    df['NodeName'] = df['fdn'].apply(lambda x: split_name(x, 'ManagedElement='))
    df['CellName'] = df['fdn'].apply(lambda x: split_name(x, 'NRCellCU='))
    df['nrcellcu_HO']  = df[counters].sum(axis=1)

    df.drop(['rdiTimeStamp', 'operator', 'pmEbsEndcPSCellChangeSuccInterSgnb',
                        'pmEbsEndcPSCellChangeSuccIntraSgnb', 'pmEbsHoExeSuccOutInterGnb',
                        'pmEbsHoExeSuccOutIntraGnb', 'fdn'], axis=1, inplace=True)
    
    return df

In [189]:
def load_nrcellrelation_ebs(nrcellrelation_ebs_filepath, date, counters):
    df = open_file(nrcellrelation_ebs_filepath)
    df = df[df['rdiTimeStamp'] == date]

    df = df[~df['pmEbsEndcPSCellChangeSuccInterSgnb'].isnull()]

    df['NodeName'] = df['fdn'].apply(lambda x: split_name(x, 'ManagedElement='))
    df['CellName'] = df['fdn'].apply(lambda x: split_name(x, 'NRCellCU='))
    df['RelName'] = df['fdn'].apply(lambda x: split_name(x, 'NRCellRelation=').replace('auto', ''))

    df['nrcellrelation_HO']  = df[counters].sum(axis=1)

    df.drop(['rdiTimeStamp', 'operator', 'pmEbsEndcPSCellChangeSuccInterSgnb',
                        'pmEbsEndcPSCellChangeSuccIntraSgnb', 'pmEbsHoExeSuccOutInterGnb',
                        'pmEbsHoExeSuccOutIntraGnb', 'fdn'], axis=1, inplace=True)

    df[['mcc', 'mnc', 'gnb', 'cellid']] = df['RelName'].str.split('_', expand=True)

    return df

In [190]:
def merge_cell_ho_counter(df_ebs_nrcellcu, df_ebs_cellrelation):
    df = df_ebs_cellrelation.merge(
        df_ebs_nrcellcu, how="left", left_on=['NodeName', 'CellName'], right_on=['NodeName', 'CellName'])

    df['arcBalance'] = df['nrcellrelation_HO']/ df['nrcellcu_HO']

    df.drop(['nrcellrelation_HO', 'nrcellcu_HO', 'mcc', 'mnc', 'RelName'], axis=1, inplace=True)

    return df

In [191]:
def merge_source_details(df_ebs_merge, df_cucp_cellcu_merged):
    df = df_ebs_merge.merge(
        df_cucp_cellcu_merged, how="left", left_on=['NodeName', 'CellName'], right_on=['NodeName', 'CellName'])

    df = df[~df['meFdn'].isnull()]

    df.rename({'CellName': 'src_nRCellCUId', 
                                'nCI': 'src_nCI',
                            'nRFrequencyRef': 'src_nRFrequencyRef'}, axis=1, inplace=True)

    df.drop(columns=['NodeName', 'meFdn', 'parentRefId', 'refId', 'cellLocalId',
                                    'GNBID', 'GNBID_LENGTH', 'MCC', 'MNC'], inplace=True)
    
    return df

In [192]:
def merge_target_details(df_ebs_source_merge, df_cucp_cellcu_merged):
    df = df_ebs_source_merge.merge(
        df_cucp_cellcu_merged, how="left", left_on=['gnb', 'cellid'], right_on=['GNBID', 'cellLocalId'])

    df.rename({'gnb': 'tgt_gNBId', 'CellName': 'tgt_nRCellCUId',
                                'nCI': 'tgt_nCI', 'GNBID_LENGTH': 'tgt_gNBIdLength', 
                            'nRFrequencyRef': 'tgt_nRFrequencyRef'}, axis=1, inplace=True)

    df.drop(columns=['meFdn', 'parentRefId', 'refId', 'cellid', 
                                    'cellLocalId', 'GNBID', 'MCC', 'MNC', 'NodeName'], inplace=True)

    df = df[~df['tgt_nCI'].isnull()]

    return df

In [193]:
def create_capacity_data(capacity_filepath, date):
    df = open_file(capacity_filepath)
    df = df[['fdn', 'rdiTimeStamp', 'pmMacRBSymUsedPdschTypeA', 'pmMacRBSymAvailDl']]
    df = df[df['rdiTimeStamp'] == date]
    df.rename({'rdiTimeStamp': 'timestamp'}, axis=1, inplace=True)

    return df

In [194]:
def create_TMO_data_sample(source_path, output_path, date):
    print('Generating CAD input data for ' + date)
    df_cucp_cellcu_merged = load_merge_gnbcucp_cellcu(source_path + '/cm/gnbcucpfunction.csv', source_path + '/cm/nrcell_cu.csv')
    write_file(df_cucp_cellcu_merged, output_path + '/cucp_cellcu_merge.csv')

    counters = ['pmEbsEndcPSCellChangeSuccInterSgnb', 'pmEbsEndcPSCellChangeSuccIntraSgnb', 
        'pmEbsHoExeSuccOutInterGnb', 'pmEbsHoExeSuccOutIntraGnb']

    df_ebs_nrcellcu = load_nrcellcu_ebs(source_path + '/ebs_NRCellCU_08_week3.csv', date, counters)

    df_ebs_cellrelation = load_nrcellrelation_ebs(source_path + '/ebs_NRCellRelation_08_week3.csv', date, counters)

    df_ebs_merge = merge_cell_ho_counter(df_ebs_nrcellcu, df_ebs_cellrelation)
    df_ebs_source_merge = merge_source_details(df_ebs_merge, df_cucp_cellcu_merged)
    df_ebs_source_merge = merge_target_details(df_ebs_source_merge, df_cucp_cellcu_merged)
    write_file(df_ebs_source_merge, output_path + '/coverage_data_' + date + '.csv')

    df_capacity = create_capacity_data(source_path + '/nrcellDU_08_week3.csv', date)
    write_file(df_capacity, output_path + '/capacity_data_' + date + '.csv')
    print('Data generation complete \n')


